# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from ge import LINE

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def combine_embedding(data,embeddings):
    i=0
    X = []
    for node1,node2 in data:
        X.append(np.concatenate((data[i],embeddings[int(node1)],embeddings[int(node2)])))
   # print(embeddings[str(int(data[0]))])
        i+=1
    return X

In [4]:
all_ap = []
all_auc = []

filename = 'texas'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']

    ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))
    
    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    
    train_df = train_df.values
    test_pos_df = test_df.values
    df_neg = df_neg.values
    
    model = LINE(train_G, embedding_size=12, order='all')
    model.train(batch_size=128, epochs=50, verbose=2)
    
    LINE_embeddings = model.get_embeddings()
    
    train_df = combine_embedding(train_df, LINE_embeddings)
    test_pos_df = combine_embedding(test_pos_df, LINE_embeddings)
    df_neg = combine_embedding(df_neg, LINE_embeddings)
    
    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)
    
    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_pos_df))
    y_test_neg = np.zeros(len(X_test_neg))
    
    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_pos_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))
    
    clf = RandomForestClassifier(n_estimators=400)
    clf.fit(X_train, y_train)
    
    predict_Y = clf.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
12/12 - 0s - loss: 1.3866 - first_order_loss: 0.6935 - second_order_loss: 0.6931
Epoch 2/50
12/12 - 0s - loss: 1.3858 - first_order_loss: 0.6932 - second_order_loss: 0.6927
Epoch 3/50
12/12 - 0s - loss: 1.3850 - first_order_loss: 0.6930 - second_order_loss: 0.6919
Epoch 4/50
12/12 - 0s - loss: 1.3838 - first_order_loss: 0.6930 - second_order_loss: 0.6907
Epoch 5/50
12/12 - 0s - loss: 1.3817 - first_order_loss: 0.6930 - second_order_loss: 0.6887
Epoch 6/50
12/12 - 0s - loss: 1.3787 - first_order_loss: 0.6929 - second_order_loss: 0.6858
Epoch 7/50
12/12 - 0s - loss: 1.3741 - first_order_loss: 0.6928 - second_order_loss: 0.6813
Epoch 8/50
12/12 - 0s - loss: 1.3681 - first_order_loss: 0.6928 - second_order_loss:

Epoch 27/50
12/12 - 0s - loss: 1.0921 - first_order_loss: 0.6870 - second_order_loss: 0.4051
Epoch 28/50
12/12 - 0s - loss: 1.0766 - first_order_loss: 0.6870 - second_order_loss: 0.3896
Epoch 29/50
12/12 - 0s - loss: 1.0673 - first_order_loss: 0.6883 - second_order_loss: 0.3789
Epoch 30/50
12/12 - 0s - loss: 1.0582 - first_order_loss: 0.6871 - second_order_loss: 0.3711
Epoch 31/50
12/12 - 0s - loss: 1.0513 - first_order_loss: 0.6877 - second_order_loss: 0.3635
Epoch 32/50
12/12 - 0s - loss: 1.0360 - first_order_loss: 0.6859 - second_order_loss: 0.3501
Epoch 33/50
12/12 - 0s - loss: 1.0280 - first_order_loss: 0.6848 - second_order_loss: 0.3432
Epoch 34/50
12/12 - 0s - loss: 1.0198 - first_order_loss: 0.6846 - second_order_loss: 0.3352
Epoch 35/50
12/12 - 0s - loss: 1.0138 - first_order_loss: 0.6841 - second_order_loss: 0.3296
Epoch 36/50
12/12 - 0s - loss: 1.0034 - first_order_loss: 0.6836 - second_order_loss: 0.3198
Epoch 37/50
12/12 - 0s - loss: 1.0013 - first_order_loss: 0.6848 - sec

Epoch 15/50
12/12 - 0s - loss: 1.2738 - first_order_loss: 0.6921 - second_order_loss: 0.5817
Epoch 16/50
12/12 - 0s - loss: 1.2580 - first_order_loss: 0.6919 - second_order_loss: 0.5662
Epoch 17/50
12/12 - 0s - loss: 1.2381 - first_order_loss: 0.6918 - second_order_loss: 0.5463
Epoch 18/50
12/12 - 0s - loss: 1.2234 - first_order_loss: 0.6915 - second_order_loss: 0.5320
Epoch 19/50
12/12 - 0s - loss: 1.2071 - first_order_loss: 0.6913 - second_order_loss: 0.5158
Epoch 20/50
12/12 - 0s - loss: 1.1891 - first_order_loss: 0.6910 - second_order_loss: 0.4981
Epoch 21/50
12/12 - 0s - loss: 1.1723 - first_order_loss: 0.6909 - second_order_loss: 0.4814
Epoch 22/50
12/12 - 0s - loss: 1.1548 - first_order_loss: 0.6903 - second_order_loss: 0.4645
Epoch 23/50
12/12 - 0s - loss: 1.1425 - first_order_loss: 0.6898 - second_order_loss: 0.4527
Epoch 24/50
12/12 - 0s - loss: 1.1249 - first_order_loss: 0.6898 - second_order_loss: 0.4350
Epoch 25/50
12/12 - 0s - loss: 1.1131 - first_order_loss: 0.6891 - sec

Epoch 3/50
12/12 - 0s - loss: 1.3844 - first_order_loss: 0.6930 - second_order_loss: 0.6913
Epoch 4/50
12/12 - 0s - loss: 1.3827 - first_order_loss: 0.6929 - second_order_loss: 0.6899
Epoch 5/50
12/12 - 0s - loss: 1.3804 - first_order_loss: 0.6929 - second_order_loss: 0.6875
Epoch 6/50
12/12 - 0s - loss: 1.3766 - first_order_loss: 0.6928 - second_order_loss: 0.6838
Epoch 7/50
12/12 - 0s - loss: 1.3715 - first_order_loss: 0.6928 - second_order_loss: 0.6788
Epoch 8/50
12/12 - 0s - loss: 1.3656 - first_order_loss: 0.6927 - second_order_loss: 0.6728
Epoch 9/50
12/12 - 0s - loss: 1.3570 - first_order_loss: 0.6926 - second_order_loss: 0.6644
Epoch 10/50
12/12 - 0s - loss: 1.3463 - first_order_loss: 0.6925 - second_order_loss: 0.6538
Epoch 11/50
12/12 - 0s - loss: 1.3358 - first_order_loss: 0.6924 - second_order_loss: 0.6433
Epoch 12/50
12/12 - 0s - loss: 1.3234 - first_order_loss: 0.6922 - second_order_loss: 0.6312
Epoch 13/50
12/12 - 0s - loss: 1.3076 - first_order_loss: 0.6921 - second_ord

Epoch 41/50
12/12 - 0s - loss: 0.9680 - first_order_loss: 0.6798 - second_order_loss: 0.2881
Epoch 42/50
12/12 - 0s - loss: 0.9657 - first_order_loss: 0.6797 - second_order_loss: 0.2860
Epoch 43/50
12/12 - 0s - loss: 0.9650 - first_order_loss: 0.6823 - second_order_loss: 0.2827
Epoch 44/50
12/12 - 0s - loss: 0.9553 - first_order_loss: 0.6776 - second_order_loss: 0.2777
Epoch 45/50
12/12 - 0s - loss: 0.9549 - first_order_loss: 0.6762 - second_order_loss: 0.2787
Epoch 46/50
12/12 - 0s - loss: 0.9501 - first_order_loss: 0.6756 - second_order_loss: 0.2745
Epoch 47/50
12/12 - 0s - loss: 0.9483 - first_order_loss: 0.6788 - second_order_loss: 0.2694
Epoch 48/50
12/12 - 0s - loss: 0.9407 - first_order_loss: 0.6772 - second_order_loss: 0.2635
Epoch 49/50
12/12 - 0s - loss: 0.9413 - first_order_loss: 0.6767 - second_order_loss: 0.2647
Epoch 50/50
12/12 - 0s - loss: 0.9437 - first_order_loss: 0.6792 - second_order_loss: 0.2645
AP：  0.6816770186335404
AUC SCORE:  0.7321428571428571
Epoch 1/50
12/1

Epoch 29/50
12/12 - 0s - loss: 1.0687 - first_order_loss: 0.6873 - second_order_loss: 0.3814
Epoch 30/50
12/12 - 0s - loss: 1.0592 - first_order_loss: 0.6870 - second_order_loss: 0.3722
Epoch 31/50
12/12 - 0s - loss: 1.0490 - first_order_loss: 0.6880 - second_order_loss: 0.3610
Epoch 32/50
12/12 - 0s - loss: 1.0409 - first_order_loss: 0.6859 - second_order_loss: 0.3550
Epoch 33/50
12/12 - 0s - loss: 1.0335 - first_order_loss: 0.6847 - second_order_loss: 0.3489
Epoch 34/50
12/12 - 0s - loss: 1.0243 - first_order_loss: 0.6855 - second_order_loss: 0.3387
Epoch 35/50
12/12 - 0s - loss: 1.0195 - first_order_loss: 0.6869 - second_order_loss: 0.3325
Epoch 36/50
12/12 - 0s - loss: 1.0078 - first_order_loss: 0.6846 - second_order_loss: 0.3232
Epoch 37/50
12/12 - 0s - loss: 1.0069 - first_order_loss: 0.6829 - second_order_loss: 0.3239
Epoch 38/50
12/12 - 0s - loss: 1.0007 - first_order_loss: 0.6837 - second_order_loss: 0.3170
Epoch 39/50
12/12 - 0s - loss: 0.9938 - first_order_loss: 0.6830 - sec